# Relax Inc. Takehome Assignment

In [596]:
#Import packages
import pandas as pd
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [492]:
#Load user data
users = pd.read_csv('takehome_users.csv')
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [493]:
#Load engagement data
engagement = pd.read_csv('takehome_user_engagement.csv')
engagement.head()

,time_stamp,user_id,visited
0,4/22/2014 3:53,1,1
1,11/15/2013 3:45,2,1
2,11/29/2013 3:45,2,1
3,12/9/2013 3:45,2,1
4,12/25/2013 3:45,2,1


In [494]:
#Engagement data shape
engagement.shape

(207917, 3)

In [496]:
#Convert 'timestamp' column to datetime
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'])

In [497]:
#Sort engagement table values by 'timestamp' column
engagement = engagement.sort_values(by=['time_stamp'])

In [498]:
#Reset engagement table index
engagement = engagement.reset_index(drop=True)
engagement.head()

,time_stamp,user_id,visited
0,2012-05-31 08:20:00,10012,1
1,2012-05-31 15:47:00,3428,1
2,2012-05-31 17:19:00,9899,1
3,2012-05-31 21:58:00,1693,1
4,2012-06-01 00:17:00,6102,1


In [499]:
#Create column with year number followed by week of the year number
engagement['week_number'] = engagement['time_stamp'].dt.strftime('%Y%U')

In [500]:
engagement.head()

,time_stamp,user_id,visited,week_number
0,2012-05-31 08:20:00,10012,1,201222
1,2012-05-31 15:47:00,3428,1,201222
2,2012-05-31 17:19:00,9899,1,201222
3,2012-05-31 21:58:00,1693,1,201222
4,2012-06-01 00:17:00,6102,1,201222


In [501]:
#Get data types of engagement table
engagement.dtypes

time_stamp     datetime64[ns]
user_id                 int64
visited                 int64
week_number            object
dtype: object

In [502]:
#Convert week number column to integer
engagement['week_number'] = [int(x) for x in engagement['week_number']]

In [503]:
#Create day of week column starting with Monday = 0 and ending with Sunday = 6
engagement['weekday'] = engagement['time_stamp'].dt.strftime('%w')

In [504]:
engagement.head()

,time_stamp,user_id,visited,week_number,weekday
0,2012-05-31 08:20:00,10012,1,201222,4
1,2012-05-31 15:47:00,3428,1,201222,4
2,2012-05-31 17:19:00,9899,1,201222,4
3,2012-05-31 21:58:00,1693,1,201222,4
4,2012-06-01 00:17:00,6102,1,201222,5


In [505]:
#Create new week number column by dense ranking the original week number column
engagement['week_num'] = engagement['week_number'].rank(method='dense').astype(int)
engagement.head()

,time_stamp,user_id,visited,week_number,weekday,week_num
0,2012-05-31 08:20:00,10012,1,201222,4,1
1,2012-05-31 15:47:00,3428,1,201222,4,1
2,2012-05-31 17:19:00,9899,1,201222,4,1
3,2012-05-31 21:58:00,1693,1,201222,4,1
4,2012-06-01 00:17:00,6102,1,201222,5,1


In [332]:
#Convert to csv to be querried with Spark
engagement.to_csv('engagement.csv')

**Start Spark Session**

In [333]:
#Open engagement table with spark
df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('engagement.csv')

In [334]:
#Create temporary view of engagement table
df.createOrReplaceTempView('grouping')

In [367]:
#Quert to get the week day and week number for each login for every user that logged in
df_sql = spark.sql('with s as (select user_id, week_num, weekday, count(weekday) as login_count\
                    from grouping \
                    group by rollup(user_id, week_num, weekday) \
                    order by user_id, week_num, weekday) \
                    select * \
                    from s \
                    where user_id is not NULL and week_num is not NULL and weekday is not NULL')
                
df_sql.show(10)

+-------+--------+-------+-----------+
|user_id|week_num|weekday|login_count|
+-------+--------+-------+-----------+
|      1|     102|      2|          1|
|      2|      78|      5|          1|
|      2|      80|      5|          1|
|      2|      82|      1|          1|
|      2|      84|      3|          1|
|      2|      85|      2|          1|
|      2|      87|      3|          1|
|      2|      91|      1|          1|
|      2|      91|      6|          1|
|      2|      92|      0|          1|
+-------+--------+-------+-----------+
only showing top 10 rows



In [368]:
#Create new temp view
df_sql.createOrReplaceTempView('grouping')

In [394]:
#Only the user_id and week_num columns were necessary, so query for just those two
df_sql = spark.sql('select user_id, week_num \
                    from grouping')
                
df_sql.show(10)

+-------+--------+
|user_id|week_num|
+-------+--------+
|      1|     102|
|      2|      78|
|      2|      80|
|      2|      82|
|      2|      84|
|      2|      85|
|      2|      87|
|      2|      91|
|      2|      91|
|      2|      92|
+-------+--------+
only showing top 10 rows



In [544]:
#Convert querried table to a pandas dataframe for further aggregation
engagement = df_sql.select("*").toPandas()

In [545]:
engagement.head()

,user_id,week_num
0,1,102
1,2,78
2,2,80
3,2,82
4,2,84


In [546]:
#Get weekly login count for each week for every user than logged in
count_series = engagement.groupby(['user_id', 'week_num']).size()
new_df = count_series.to_frame(name = 'size').reset_index()

In [547]:
new_df.head(10)

,user_id,week_num,size
0,1,102,1
1,2,78,1
2,2,80,1
3,2,82,1
4,2,84,1
5,2,85,1
6,2,87,1
7,2,91,2
8,2,92,2
9,2,93,1


In [548]:
#If the number of logins on separate days within a week is greater than three,
#the user could be considered adopted and marked with a 1. Else, the user is not considered adopted.
l = []
for x in new_df['size']:
    if x >= 3:
        l.append(1)
    else:
        l.append(0)

In [549]:
new_df['adopted'] = l

In [550]:
adopt_id = new_df[new_df['adopted']==1]

In [551]:
l = []
for x in adopt_id['user_id']:
    l.append(x)

In [552]:
#Get only unique adopted user values
adopted_id = sorted(list(set(l)))

In [554]:
#Create adopted column for users table
l = []
for x in users['object_id']:
    if x in adopted_id:
        l.append(1)
    else:
        l.append(0)

In [555]:
users['adopted'] = l

In [556]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,invited_a_user,adopted
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803,1,0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316,1,0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525,1,0
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151,1,0
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240,1,0


**Exploratory Data Analysis**

In [557]:
#Get unique value count for each variable in users table
users.nunique()

object_id                     12000
creation_time                 11930
name                          11355
email                         11980
creation_source                   5
last_session_creation_time     8821
opted_in_to_mailing_list          2
enabled_for_marketing_drip        2
org_id                          417
invited_by_user_id             2565
invited_a_user                    1
adopted                           2
dtype: int64

In [558]:
#Get users shape
users.shape

(12000, 12)

In [559]:
#Get users info
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          12000 non-null  int64  
 10  invited_a_user              12000 non-null  int64  
 11  adopted                     12000 non-null  int64  
dtypes: float64(1), int64(7), object(4)
memory usage: 1.1+ MB


In [567]:
#Fill missing values in 'invited_by_user_id' column with zero
users['invited_by_user_id'] = users['invited_by_user_id'].fillna(0)

In [572]:
l=[]
for x in users['invited_by_user_id']:
    if x != 0:
        l.append(1)
    else:
        l.append(0)
users['invited_by_a_user'] = l

In [576]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted,invited_by_a_user
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803,0,1
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316,0,1
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525,0,1
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151,0,1
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240,0,1


In [577]:
#Drop unnecessary columns
df = users.drop(columns=['object_id', 'creation_time', 'name', 'email', 'last_session_creation_time', 'org_id', 'invited_by_user_id'])

In [578]:
df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,adopted,invited_by_a_user
0,GUEST_INVITE,1,0,0,1
1,ORG_INVITE,0,0,0,1
2,ORG_INVITE,0,0,0,1
3,GUEST_INVITE,0,0,0,1
4,GUEST_INVITE,0,0,0,1


In [580]:
#Get dummies for the 'creation_source' column
dummy_df = pd.get_dummies(df['creation_source'])
dummy_df.head()

,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1,0,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


In [582]:
#Concatenate dummy variables with df
df = pd.concat([df, dummy_df], axis=1, sort=False)

In [584]:
#Drop 'creation_source' column
df = df.drop(columns='creation_source')

In [585]:
df.head()

,opted_in_to_mailing_list,enabled_for_marketing_drip,adopted,invited_by_a_user,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1,0,0,1,1,0,0,0,0
1,0,0,0,1,0,1,0,0,0
2,0,0,0,1,0,1,0,0,0
3,0,0,0,1,1,0,0,0,0
4,0,0,0,1,1,0,0,0,0


**Create Random Forest**

In [587]:
#create predicting and target sets
X = df.drop(columns='adopted')
y = df['adopted']

In [588]:
#Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [592]:
#Create random forest classifier
clf = RandomForestClassifier().fit(X_train, y_train.values.ravel())

In [597]:
#Get accuracy score of fitted random forest model
predictions_rf = clf.predict(X_test)
print(accuracy_score(y_test, predictions_rf))

0.8795454545454545


In [598]:
#Get feature importances for random forest model
important_features = pd.Series(data=clf.feature_importances_,index=X.columns)
print(important_features.sort_values(ascending=False,inplace=True))

None


In [599]:
#Show feature importances from random forest model
important_features.head(15)

PERSONAL_PROJECTS             0.341265
opted_in_to_mailing_list      0.129830
GUEST_INVITE                  0.125387
enabled_for_marketing_drip    0.115446
SIGNUP_GOOGLE_AUTH            0.103726
ORG_INVITE                    0.071084
SIGNUP                        0.064354
invited_by_a_user             0.048909
dtype: float64

After finding out which users were adopted using the engagement table, I processed the data and fit it to a random forest model. Once the model was fit, I got the accuract score and a list of each feature by order of importance. Based on the insights obtained from the random forest model, being invited to join another user's workspace was a strong predictor of whether a user was adopted. Mailing lists and guest invites were also relatively strong predictors and signups were not effective.